In [ ]:
import csv

In [ ]:
import os

DATADIR = "DataWranglingwithMongoDB/"
DATAFILE = "beatles-diskography.csv"
DATAFILE = os.path.join(DATADIR, DATAFILE)

def parse_file(datafile):
    data = []
    num = 0
    with open(datafile, "r") as f:
        for line in f:
            line = line.strip('\n') 
            if num == 0:
                Keys = line.split(',')
            if num > 0 and num < 11:
                Values = line.split(',')
                beatles = dict(zip(Keys, Values))
                data.append(beatles) 
            num += 1
    return data

In [ ]:
data = parse_file(DATAFILE)

In [ ]:
print(data)

In [ ]:
"""
Your task is as follows:
- read the provided Excel file
- find and return the min, max and average values for the COAST region
- find and return the time value for the min and max entries
- the time values should be returned as Python tuples

Please see the test function for the expected return format
"""

import xlrd
import os
import numpy as np

from zipfile import ZipFile
datafile = "2013_ERCOT_Hourly_Load_Data.xls"
datadir = "./DataWranglingwithMongoDB/"

datafile = os.path.join(datadir,datafile)

def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile.rsplit('.',1)[0]), 'r') as myzip:
        myzip.extractall(datadir)
       
def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    ### example on how you can get the data
    #sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]

    ### other useful methods:
    print("\nROWS, COLUMNS, and CELLS:")
    print("Number of rows in the sheet:")
    print(sheet.nrows)
    print("Type of data in cell (row 3, col 2):") 
    print(sheet.cell_type(3, 2))
    print ("Value in cell (row 3, col 2):", )
    print(sheet.cell_value(3, 2))
    print("Get a slice of values in column 2 - COAST from rows 1-3:")

    coast_vals = sheet.col_values(1, start_rowx=1)
    
    print
    print("\nDATES:")
    print("Type of data in cell (row 1, col 0):")
    print(sheet.cell_type(1, 0))
    exceltime = sheet.cell_value(1, 0)
    print("Time in Excel format:",)
    print(exceltime)
    print("Convert time to a Python datetime tuple, from the Excel float:")
    print(xlrd.xldate_as_tuple(exceltime, 0))
    
    
    data = {
            'maxtime': (0, 0, 0, 0, 0, 0),
            'maxvalue': 0,
            'mintime': (0, 0, 0, 0, 0, 0),
            'minvalue': 0,
            'avgcoast': 0
    }
    coast_vals = np.array(coast_vals)
    
    data['avgcoast'] = np.average(coast_vals)
    data['maxvalue'] = np.max(coast_vals)
    data['minvalue'] = np.min(coast_vals)
    
    maxidx = coast_vals.argmax()
    minidx = coast_vals.argmin()
    maxtime = xlrd.xldate_as_tuple(sheet.cell_value(maxidx + 1, 0),0)
    mintime = xlrd.xldate_as_tuple(sheet.cell_value(minidx + 1, 0),0)
    
    data['maxtime'] = maxtime
    data['mintime'] = mintime
    
    print(data)
    return data

open_zip(datafile)
data = parse_file(datafile)
#print('{0}.zip'.format(datafile))

In [ ]:
def test():
    open_zip(datafile)
    data = parse_file(datafile)

    assert data['maxtime'] == (2013, 8, 13, 17, 0, 0)
    assert round(data['maxvalue'], 10) == round(18779.02551, 10)


test()

In [2]:
# To experiment with this code freely you will have to run this code locally.
# Take a look at the main() function for an example of how to use the code.
# We have provided example json output in the other code editor tabs for you to
# look at, but you will not be able to run any queries through our UI.
import json
import requests


BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"

# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    # This is the main function for making queries to the musicbrainz API.
    # A json document should be returned by the query.
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print("requesting", r.url)
    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    # This adds an artist name to the query parameters before making
    # an API call to the function above.
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    # After we get our output, we can format it to be more readable
    # by using this function.
    if type(data) == dict:
        print(json.dumps(data, indent=indent, sort_keys=True))
    else:
        print(data)


def main():
    '''
    Modify the function calls and indexing below to answer the questions on
    the next quiz. HINT: Note how the output we get from the site is a
    multi-level JSON document, so try making print statements to step through
    the structure one level at a time or copy the output to a separate output
    file.
    '''
    results = query_by_name(ARTIST_URL, query_type["simple"], "Queen")
    pretty_print(results)

    artist_id = results["artists"][1]["id"]
    print ("\nARTIST:")
    pretty_print(results["artists"][1])

    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]
    print("\nONE RELEASE:")
    pretty_print(releases[0], indent=2)
    release_titles = [r["title"] for r in releases]

    print("\nALL TITLES:")
    for t in release_titles:
        print (t)


#if __name__ == '__main__':
#    main()

In [3]:
results = query_by_name(ARTIST_URL, query_type["simple"], "One Direction")
print([ artist["name"] for artist in results["artists"] ])
#pretty_print(results)


requesting http://musicbrainz.org/ws/2/artist/?fmt=json&query=artist%3AOne+Direction
['One Direction', 'Direction', 'Direction', 'New Direction', 'Second Direction', 'Loose Direction', 'The Direction', 'Lost Direction', 'Audio Direction', 'Disco Direction', 'No Direction', '7th Direction', 'Direction X', 'Direction Indicator', 'No Direction', 'Right Direction', '高柳昌行 New Direction', 'New Direction for the Arts', 'The New Transit Direction', 'The Beat Direction', 'Found My Direction', 'Tensor & Re-Direction', 'Moving in The Right Direction', 'Chorale et assemblée sous la direction de Jean Zune', 'Orchestre Symphonique National direction Wal-Berg']


In [4]:
results["artists"][0]

{'area': {'id': '8a754a16-0027-3a29-b6d7-2b40ea0481ed',
  'name': 'United Kingdom',
  'sort-name': 'United Kingdom'},
 'begin-area': {'id': 'f03d09b3-39dc-4083-afd6-159e3f0d462f',
  'name': 'London',
  'sort-name': 'London'},
 'country': 'GB',
 'disambiguation': 'English-Irish boy band formed in 2010',
 'id': '1a425bbd-cca4-4b2c-aeb7-71cb176c828a',
 'life-span': {'begin': '2010-07', 'ended': None},
 'name': 'One Direction',
 'score': '100',
 'sort-name': 'One Direction',
 'tags': [{'count': 2, 'name': 'pop'},
  {'count': 1, 'name': 'power pop'},
  {'count': 1, 'name': 'dance-pop'},
  {'count': 1, 'name': 'pop rock'},
  {'count': 1, 'name': 'folk pop'},
  {'count': 2, 'name': 'boy band'}],
 'type': 'Group'}

In [12]:
#!/usr/bin/env python
"""
Your task is to process the supplied file and use the csv module to extract data from it.
The data comes from NREL (National Renewable Energy Laboratory) website. Each file
contains information from one meteorological station, in particular - about amount of
solar and wind energy for each hour of day.

Note that the first line of the datafile is neither data entry, nor header. It is a line
describing the data source. You should extract the name of the station from it.

The data should be returned as a list of lists (not dictionaries).
You can use the csv modules "reader" method to get data in such format.
Another useful method is next() - to get the next line from the iterator.
You should only change the parse_file function.
"""
import csv
import os

DATADIR = "./data/"
DATAFILE = "745090.csv"



def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'rt') as f:
        reader = csv.reader(f)
        for line in reader:
            if line == 0:
                name = line[0].splt()[2]
                next(f)
            print(line)
    #    pass
    # Do not change the line below
    return (name, data)

"""
def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    name, data = parse_file(datafile)

    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"
    assert data[0][1] == "01:00"
    assert data[2][0] == "01/01/2005"
    assert data[2][5] == "2"
"""

#if __name__ == "__main__":
#    test()



'\ndef test():\n    datafile = os.path.join(DATADIR, DATAFILE)\n    name, data = parse_file(datafile)\n\n    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"\n    assert data[0][1] == "01:00"\n    assert data[2][0] == "01/01/2005"\n    assert data[2][5] == "2"\n'

In [13]:
datafile = os.path.join(DATADIR,DATAFILE)
parse_file(datafile)

['745090', 'MOUNTAIN VIEW MOFFETT FLD NAS', 'CA', '-8.0', '37.400', '-122.050', '12']
['Date (MM/DD/YYYY)', 'Time (HH:MM)', 'ETR (W/m^2)', 'ETRN (W/m^2)', 'GHI (W/m^2)', 'GHI source', 'GHI uncert (%)', 'DNI (W/m^2)', 'DNI source', 'DNI uncert (%)', 'DHI (W/m^2)', 'DHI source', 'DHI uncert (%)', 'GH illum (lx)', 'GH illum source', 'Global illum uncert (%)', 'DN illum (lx)', 'DN illum source', 'DN illum uncert (%)', 'DH illum (lx)', 'DH illum source', 'DH illum uncert (%)', 'Zenith lum (cd/m^2)', 'Zenith lum source', 'Zenith lum uncert (%)', 'TotCld (tenths)', 'TotCld source', 'TotCld uncert (code)', 'OpqCld (tenths)', 'OpqCld source', 'OpqCld uncert (code)', 'Dry-bulb (C)', 'Dry-bulb source', 'Dry-bulb uncert (code)', 'Dew-point (C)', 'Dew-point source', 'Dew-point uncert (code)', 'RHum (%)', 'RHum source', 'RHum uncert (code)', 'Pressure (mbar)', 'Pressure source', 'Pressure uncert (code)', 'Wdir (degrees)', 'Wdir source', 'Wdir uncert (code)', 'Wspd (m/s)', 'Wspd source', 'Wspd uncert

('', [])